In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
from fp_gen import KlekFPGenerator, MACCSFPGenerator, SubFPGenerator
from DataProc import *
import pandas.api.types as ptypes
import matplotlib.pyplot as plt
import seaborn as sns
import rdkit
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem, DataStructs
sys.path.append('..')

In [5]:
generator = KlekFPGenerator(n_jobs=os.cpu_count())
data = pd.read_csv('../original_datasets/artificial/d2_artificial.csv', chunksize=500, names=['SMILES'])

for i, chunk in enumerate(data):
    chunk = chunk[chunk['SMILES'].apply(could_be_valid)]
    mols = list(map(Chem.MolFromSmiles, chunk['SMILES']))
    fps = generator.transform(mols)
    chunk['fps'] = list(map(sparse_to_dense, fps))
    if i == 0:
        chunk.to_csv('../original_datasets/artificial/d2_klek_artificial.csv', index=False)
    else: # append if already exists, otherwise without else firts chunk will be written twice
        with open('../original_datasets/artificial/d2_klek_artificial.csv', 'a') as f:
            chunk.to_csv(f, header=False, index=False)

[06:17:59] SMILES Parse Error: syntax error while parsing: SMILES
[06:17:59] SMILES Parse Error: Failed parsing SMILES 'SMILES' for input: 'SMILES'
C:\Users\matit\AppData\Local\Temp\ipykernel_16680\557390353.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['fps'] = list(map(sparse_to_dense, fps))


In [3]:
proteins = ['5ht1a','5ht7','beta2','d2','h1']
for protein in proteins:
    data = DataProcessor(protein)
    data.load_data()
    data.remove_missing()
    data.remove_duplicates()
    data.add_classification()
    data.write_cleaned()
    data.write_parquet()

Loaded data for 5ht1a protein
Loaded data for 5ht7 protein
Loaded data for beta2 protein
Loaded data for d2 protein
Loaded data for h1 protein


In [38]:
def csv_to_parquet(read_path, write_path):
    data = pd.read_csv(read_path, sep=',')
    data.to_parquet(write_path)

read_paths = [f'../original_datasets/klek_clean/{x}_klek_balanced.csv' for x in proteins]
write_paths = [f'../original_datasets/parquet_clean/{x}_klek_balanced.parquet' for x in proteins]
    
for read_path, write_path in zip(read_paths, write_paths):
    print(read_path, write_path)
    csv_to_parquet(read_path, write_path)

../original_datasets/klek_clean/5ht1a_klek_balanced.csv ../original_datasets/parquet_clean/5ht1a_klek_balanced.parquet
../original_datasets/klek_clean/5ht7_klek_balanced.csv ../original_datasets/parquet_clean/5ht7_klek_balanced.parquet
../original_datasets/klek_clean/beta2_klek_balanced.csv ../original_datasets/parquet_clean/beta2_klek_balanced.parquet
../original_datasets/klek_clean/d2_klek_balanced.csv ../original_datasets/parquet_clean/d2_klek_balanced.parquet
../original_datasets/klek_clean/h1_klek_balanced.csv ../original_datasets/parquet_clean/h1_klek_balanced.parquet
